In [1]:
import numpy as np
import pandas as pd
import random

# Minimal Spanning Three

In this example, I will demonstrate three different scenarios, just to show how many seemingly different problems may be theoretically identical, and can be solved using the same procedure.

<img src="05 - Minimal Spanning Three image 01.png">

Possible problems could be:
1. Outside of Kristiansand, several fishing boats a stricken with fair as they hear on the radio that an extremly thick fog will cover their fishing area very soon. To avoid anyone getting lost in the fog while fishing, they decide that everyone should tie themselfes to a different boat. However, they have a limited amount of rope, and each fishing boat is reluctant to move away from their designated fishing spot. How much rope is the minimum length needed to tie the bots together so that all the boats are connected in eachother through at leat one neighbouring boat?
2. A farmer from Søgne wants to set up a watering system at his farm. He knows where the irrigation sprinkler units should be placed, but he doesn't want to spend more money than he has to on watering hoses. Each sprinkler system can be connected to multiple other sprinklers. How should he connect them to achieve the total minimum length of watering hoses?
3. Five new oil fields are discovered in the North Sea by Equinor. They are investigating if it is possible to connect the reservoirs rather than having the oil platform move to each reservoir separatly. This could save a lot of cost if the pipes needed to pull this off was low enough. How great is the minimum distance that needs to be covered?
 
This problem can be formulated as a MILP problem. However, it turns out we can use a much simpler approach called Prim's algorithm. Prim's algorithm is a so-called greedy algorithm. These types of algorithms are famous for yielding fast results that often satisfy the constraints, but without being optimal. However, for the minimum spanning tree problem, this, or alternativly Kruskal's algorithm, are actually able to find the optimal solution 

We can re-draw the problem as a graph, where each node corresponds to the units we want to connect to eachother, and the edges correspond to the distance between them. The numbers are unitsless in the graph, but can thought of as meters, kilometers, nautical miles or some other unit depending on what best fits the problem.

<img src="05 - Minimal Spanning Three image 02.png">

## Prepare the data

In [2]:
# Constants
Nodes = ['A','B','C','D','E','F']

In [3]:
# Maximum flow
data = [
    [np.nan,     23, np.nan, np.nan,     27,     73],
    [    23, np.nan,     26, np.nan,      8, np.nan],
    [np.nan,     26, np.nan,     53,     32, np.nan],
    [np.nan, np.nan,     53, np.nan,     34,     23],
    [    27,      8,     32,     34, np.nan,     24],
    [    73, np.nan, np.nan,     23,     24, np.nan]
]

graph = pd.DataFrame(data=data, index=Nodes, columns=Nodes)
graph.fillna(' ')

,A,B,C,D,E,F
A,,23,,,27,73
B,23,,26,,8,
C,,26,,53,32,
D,,,53,,34,23
E,27,8,32,34,,24
F,73,,,23,24,


## Solve the problem using Prim's Algorithm

The peseudo-code for Prim's algorithm is as follows:

```
Step 1:
Select any one node from the network (doesn't matter which one), and add the edges going out from that node to your tree of possible routes

Step 2:
Choose the smallest edge with the smallest distance from your tree of possible routes, and append it to your tree of selected routes

Step 3:
Repreat step 2 until all nodes are in your tree of selected routes
```

In [4]:
# initialize the MST and the set of selected nodes
selected_edges = set();
selected_nodes = set();

objective_value = 0

# Step 1:  select an arbitrary vertex to begin with
selected_nodes.add(random.choice(Nodes));

while len(selected_nodes) != len(Nodes):
    crossing = set();
    # for each element i in selected_nodes, add the edge (i, j) to crossing if
    # j is not in selected_nodes
    for i in selected_nodes:
        for j in Nodes:
            if j not in selected_nodes and pd.notna(graph.loc[i,j]):
                crossing.add((i, j))
    
    # Step 2: find the edge with the smallest weight in crossing
    edge = sorted(crossing, key=lambda e:graph[e[0]][e[1]])[0];
    # add this edge to selected_edges
    selected_edges.add(edge)
    # add the new node to selected_nodes
    selected_nodes.add(edge[1])
    # Add the edge to the "objective value"
    objective_value += graph.loc[edge]    
    
    print(f"Appended the edge {edge} with a length of {graph.loc[edge]:2.0f}")

print()
print(f"Total length: {objective_value:.0f}")

Appended the edge ('E', 'B') with a length of  8
Appended the edge ('B', 'A') with a length of 23
Appended the edge ('E', 'F') with a length of 24
Appended the edge ('F', 'D') with a length of 23
Appended the edge ('B', 'C') with a length of 26

Total length: 104


The answer, no matter what node is used to initialize the problem, will be the one shown in the image below. Note that if there are multiple solutions that are equally good, the solution will depend on the node you start with.

<img src="05 - Minimal Spanning Three image 03.png">